In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-28 18:37:05--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-28 18:37:05 (24.2 MB/s) - ‘minsearch.py.4’ saved [3832/3832]



In [16]:
import minsearch

In [17]:
import json

In [4]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 18:37:10--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.3’

documents.json.3    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-28 18:37:10 (62.7 MB/s) - ‘documents.json.3’ saved [658332/658332]



In [18]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [19]:
documents =[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
q = 'the course has already started, can I still enroll?'

In [22]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], 
    keyword_fields=["course"]
)

In [23]:

index.fit(documents)

In [24]:
from openai import OpenAI

In [25]:
client = OpenAI()

In [26]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUSTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [53]:
query = 'how do I run kafka?'



In [30]:
rag(query)

"To run Kafka, you need to execute the appropriate command based on your project setup and whether you're using Java or Python.\n\nFor Java:\nIn the project directory, you can run the following command:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor Python:\n1. Create a virtual environment and install the necessary packages. To do this, run:\n    ```bash\n    python -m venv env\n    source env/bin/activate\n    pip install -r path/to/requirements.txt\n    ```\n    Note: On Windows, activate the virtual environment with `env\\Scripts\\activate`.\n\n2. Ensure all Docker images are up and running.\n\nAfter setting up the environment, you can run your Kafka Python files within the virtual environment you created."

In [31]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
from elasticsearch import Elasticsearch

In [33]:
es_clent = Elasticsearch('http://localhost:9200')

In [34]:
es_clent.info()

ObjectApiResponse({'name': '9973c2eb71c4', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2LyiqGVlRjy-12KrzImlFQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [35]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

es_clent.indices.create(index = index_name, body = index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_questions/qbkFHPm3TBm0HjwPuT0IYw] already exists')

In [36]:
from tqdm.auto import tqdm

In [37]:
for doc in tqdm(documents):
    es_clent.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [56]:
query = 'I just discovered the course, can I still join it?'

In [57]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_clent.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs    

In [58]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [59]:
rag(query)

'Yes, you can still join the course even after it has started. You are eligible to submit the homeworks without registering. Just keep in mind that there will be deadlines for turning in the final projects, so try not to leave everything until the last minute.'